<h1><center>Analytics for Unstructured Data Assignment 2 - Beer Recommender System</center></h1>

<center><b>Akshat Johari: aj32864 </b></center>
<center><b>Aritra Chowdhury: ac79277 </b></center>
<center><b>Christian Alfonso: ca35748 </b></center>
<center><b>Rushiil Deshmukh: rsd949 </b></center>
<center><b>Troy Richard: tdr944 </b></center>

## Library Imports

In [1]:
# Importing all required libraries
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
stop_words = set( stopwords.words('english'))
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy
from spacy.lang.en import English 
import en_core_web_lg
import numpy as np
import string
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from selenium import webdriver

## Task A: Web Scraping

We perform Webscraping and store all the Reviews in a separate CSV file, *Reviews.csv*. We use that CSV file for data extraction and processing. Currently, we have commented out the webscraper and are providing the requisite csvs in a google drive link

In [2]:
# wd = webdriver.Firefox(executable_path=r'C:\Python\Webdriver\geckodriver.exe')
# beers = pd.DataFrame(columns = ['name', 'rating', 'link'])
# wd.get('https://www.beeradvocate.com/beer/top-rated/')
# for i in range(0,250):
#     # Getting the link to the page for each beer
#     link_path = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr[' + str(i+2) + ']/td[2]/a')[0]
#     link = link_path.get_attribute('href')
#     name_path = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr[' + str(i+2) + ']/td[2]/a/b')[0]
#     name = name_path.text
#     rating_path = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr[' + str(i+2) + ']/td[4]/b')[0]
#     rating = rating_path.text
#     beers.loc[len(beers)] = [name, rating, link]
# beers
# beers.to_csv('Beer Details.csv')

In [3]:
# # Reviews of each of the beers
# reviews = pd.DataFrame(columns = ['product_name', 'product_review', 'user_rating'])

# links = beers['link'].to_list()

# for k,i in enumerate(links):
#     wd.get(i)
#     for j in range(0,31):
#         try:
#             rating_path = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div\
#             /div[3]/div/div/div[2]/div[8]/div/div[' + str(j+1) + ']/div[2]/span[2]')[0]
#             rating = rating_path.text
#             review_path = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]\
#             /div/div/div[2]/div[8]/div/div[' + str(j+1) + ']/div[2]')[0]
#             review = review_path.text
#             review = review.split("\n\n")[2]
#             name = beers.loc[k]['name']
#             reviews.loc[len(reviews)] = [name, review, rating]
#         except:
#             pass
# reviews.to_csv('Reviews.csv')

### Reading the Reviews column for recommendation

In [4]:
url = "https://drive.google.com/file/d/10wBvRpHwLk8-aYa6yc2YpqIIrlCVjuMx/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df_reviews = pd.read_csv(path)
df_reviews = df_reviews.drop(columns='Unnamed: 0')
df_reviews = df_reviews.dropna()
df_reviews.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic. It ...,5.00
1,Kentucky Brunch Brand Stout,"Long time waiting to tick this one, and I have...",4.56
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch. It was bott...,5.00
3,Kentucky Brunch Brand Stout,Supreme maple OD! Soooo easy drinking & well-t...,5.00
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00


### Data Processing and EDA

In [5]:
# function to remove punctuations from a string, for word frequency analysis
def remove_punc(s):
    s = s.translate(str.maketrans('','',string.punctuation))
    return s

# function to remove stopwords from a string
def remove_stopwords(s):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_comments = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    return filtered_comments

# adding cleaned reviews and tokens to the reviews DataFrame
df_reviews['cleaned_review'] = df_reviews['product_review'].apply(remove_punc)
df_reviews['review_tokens'] = df_reviews['cleaned_review'].apply(remove_stopwords)
df_reviews['joined_tokens'] = df_reviews['review_tokens'].apply(lambda x: ' '.join(x))
df_reviews.head()

,product_name,product_review,user_rating,cleaned_review,review_tokens,joined_tokens
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic. It ...,5.00,2020 vintage acquired during the pandemic It w...,"[2020, vintage, acquired, pandemic, everything...",2020 vintage acquired pandemic everything id h...
1,Kentucky Brunch Brand Stout,"Long time waiting to tick this one, and I have...",4.56,Long time waiting to tick this one and I have ...,"[long, time, waiting, tick, one, say, pretty, ...",long time waiting tick one say pretty underwhe...
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch. It was bott...,5.00,This review is for the 2019 batch It was bottl...,"[review, 2019, batch, bottle, number, 43, tast...",review 2019 batch bottle number 43 tasted part...
3,Kentucky Brunch Brand Stout,Supreme maple OD! Soooo easy drinking & well-t...,5.00,Supreme maple OD Soooo easy drinking welltast...,"[supreme, maple, od, soooo, easy, drinking, we...",supreme maple od soooo easy drinking welltasty...
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00,I have now had 4 different years of KBBS and c...,"[4, different, years, kbbs, continue, feel, be...",4 different years kbbs continue feel beer real...


In [6]:
# calculating word frequency
count = []
count = pd.Series(np.concatenate([x for x in df_reviews['review_tokens']]))
word_freq = dict(nltk.FreqDist(count))

def sort_word(x, reverse = True):
    return dict(sorted(x.items(), key = lambda x: x[1], reverse = reverse))

# storing word frequency in a variable
sorted_words = sort_word(word_freq)

## Task B: Beer and User Attributes

### Beer Attributes (Dummies Cheat Sheet)

We carry out a Word Frequency Analysis on the beer attributes mentioned in the following link.
https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/ 

The attributes are stored in a CSV file, *Beer attributes.csv*.

In [7]:
# reading the beer attributes from a csv file on Google Drive
url = "https://drive.google.com/file/d/1jTxB9jpa2siCkrH-6W0m4_5IEQFQSpGf/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df_attr = pd.read_csv(path)

In [8]:
# computing word frequencies for the above beer attributes
attr_list = df_attr['Attributes'].tolist()
attr_list = [x.lower() for x in attr_list]
attr = {}
for key, values in sorted_words.items():
    if key in attr_list:
        attr[key] = values
print('Word Frequencies for the beer attributes mentioned in the Dummies Cheat Sheet:')
attr

Word Frequencies for the beer attributes mentioned in the Dummies Cheat Sheet:


{'balanced': 380,
 'complex': 290,
 'fruity': 208,
 'floral': 170,
 'hoppy': 145,
 'roasty': 138,
 'crisp': 98,
 'malty': 91,
 'aggressive': 44,
 'robust': 40,
 'diacetyl': 9,
 'estery': 1}

### User Attributes

We read the User Attributes from a separate CSV file, *User attributes.csv*.  

We assume the user attributes to be the following.

In [9]:
# reading the user attributes from a csv file
url = "https://drive.google.com/file/d/1_3DEfaAWmucxjcM6WSGRvTlDdiZPwWbo/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
user_attr = pd.read_csv(path)['Attributes'].str.lower().tolist()
print('\033[1mUser Attributes:\033[0m')
for i in user_attr:
    print(i)

User Attributes:
hoppy
complex
fruity


## Task C: Similarity Analysis

### Similarity Score for each Review

In [10]:
# joining the user attributes in a sentence for cosine similarity analysis
user_attr_str = ' '.join(user_attr)

In [11]:
df_reviews_cosine = df_reviews[['product_name','product_review','cleaned_review','joined_tokens','user_rating']]
score_list = []
count_vec = CountVectorizer()

# function to calculate cosine similarity
def cosine_sim(s):
    doc = [user_attr_str,s]
    vec_mat = count_vec.fit_transform(doc)
    cosine_similarity_matrix = cosine_similarity(vec_mat)
    df_cosine = pd.DataFrame(cosine_similarity_matrix)
    return cosine_similarity(df_cosine,df_cosine)[0,1]

# calculating similarity score and storing it in a DataFrame
df_reviews_cosine['similarity_score'] = df_reviews_cosine['joined_tokens'].apply(cosine_sim)
df_reviews_cosine = df_reviews_cosine.sort_values(by = 'similarity_score', ascending = False)
df_reviews_cosine_final = df_reviews_cosine[['product_name','product_review','similarity_score']]
df_reviews_cosine_final = df_reviews_cosine_final.sort_values(by = 'similarity_score', ascending = False)

# storing the resultant DataFrame in a csv file
df_reviews_cosine_final.to_csv('Similarity Analysis (Task C).csv')

# displaying the top 10 reviews by similarity score
print('Top 10 Reviews by Similarity Score (Cosine Similarity):')
df_reviews_cosine_final.head(10)

Top 10 Reviews by Similarity Score (Cosine Similarity):


,product_name,product_review,similarity_score
344,Pliny The Elder,"Bottle, kindly shared by gunnar. Deep golden t...",0.699854
5827,Vicinity,"Canned 09/23/20. Pours a hazy yellow, with som...",0.539313
6014,Black & Wild,Was lucky enough to have this one at the sourc...,0.532939
4201,Saison Bernice,Cloudy straw appearance with slight white head...,0.495071
2175,Galaxy Dry Hopped Fort Point Pale Ale,"11/2014 - Wow, this is amazing; incredible aro...",0.495071
4382,Hopslam Ale,Great super hoppy beer! It was one of the firs...,0.484123
4279,Bad Boy,A fantastic big hoppy IPA that is surprisingly...,0.473866
1530,Citra,02/2015 - Pineapple and mango fosho with mild ...,0.464231
1619,Focal Banger,Shared can at regional tasting. Slightly hazy ...,0.446594
198,King Julius,"nothing but fresh, hoppy dankness coming out o...",0.446594


### Average Similarity Score for each Product

In [12]:
# reading the above output file to compute average similarity for each product
df_reviews_avg_similarity = pd.read_csv('Similarity Analysis (Task C).csv')

# calculating average similarity for each product
df_reviews_avg_similarity = df_reviews_avg_similarity.groupby('product_name').mean()[['similarity_score']]\
                                                     .reset_index()\
                                                     .sort_values(by = 'similarity_score', ascending = False)

# displaying the top 10 products by average similarity score
print('Top 10 Products by Average Similarity Score (Cosine Similarity):')
df_reviews_avg_similarity.head(10)

Top 10 Products by Average Similarity Score (Cosine Similarity):


,product_name,similarity_score
94,Ephraim,0.080040
191,Pliny The Elder,0.076789
99,Focal Banger,0.063240
21,Bad Boy,0.058161
210,Society & Solitude #6,0.057589
153,King Sue,0.055952
112,Galaxy Dry Hopped Fort Point Pale Ale,0.054713
31,Black & Wild,0.052657
92,Duck Duck Gooze,0.051493
154,King Sue - Double Dry-Hopped,0.049948


## Task D: Sentiment Analysis

### Sentiment Analysis for each Review

In [13]:
# using the Sentiment Intensity Analyzer function to compute sentiment scores for each review
sd = SentimentIntensityAnalyzer()
df_reviews_cosine['sentiment_score'] = df_reviews_cosine['cleaned_review'].apply(lambda x: sd.polarity_scores(x)['compound'])
df_reviews_sent = df_reviews_cosine[['product_name','product_review','similarity_score','sentiment_score','user_rating']]
df_reviews_sent = df_reviews_sent.sort_values(by = 'sentiment_score', ascending = False)

# displaying the top 10 reviews by sentiment score
print('Top 10 Reviews by Sentiment Score:')
df_reviews_sent.head(10)

Top 10 Reviews by Sentiment Score:


,product_name,product_review,similarity_score,sentiment_score,user_rating
2792,The Peach,one of the beers i have been after for a long ...,0.060718,0.9991,4.45
922,Westly,wow! thanks to drlovemd87 for sharing this rar...,0.000000,0.9989,4.55
1591,Nectarine Premiere,this one has been on my most wanted list for a...,0.072216,0.9985,4.45
1370,Aaron,a near perfect example of an english barleywin...,0.000000,0.9983,4.55
5874,Cellarman Barrel Aged Saison,good heavens this is good. thanks to drlovemd8...,0.000000,0.9983,4.53
5117,Clare's Thirsty Ale,been looking forward to trying this one for a ...,0.000000,0.9980,4.34
4447,Leaner,"killer beer, amazing to have in such close pro...",0.075059,0.9978,4.64
608,Bourbon County Brand Stout,Well there’s a lot of hype surrounding this be...,0.000000,0.9978,5.00
1641,Black Tuesday,Definitely one of those beers i've waited a lo...,0.000000,0.9976,5.00
3044,Consecration,This is an excellent A+++ premium Beer all the...,0.000000,0.9974,4.94


## Task E: Evaluation Scores

### Evaluation Scores for each Product

In [14]:
# calculating evaluation scores for each product
df_reviews_eval_cos = df_reviews_sent.groupby('product_name').mean()[['similarity_score','sentiment_score','user_rating']]
df_reviews_eval_cos['evaluation_score'] = df_reviews_eval_cos['similarity_score'] + df_reviews_eval_cos['sentiment_score']
df_reviews_eval_cos = df_reviews_eval_cos.sort_values(by = 'evaluation_score', ascending = False).reset_index()

# displaying the top 10 products by evaluation score
print('Top 10 Products by Evaluation Score (Cosine Similarity and Sentiment):')
df_reviews_eval_cos.head(10)

Top 10 Products by Evaluation Score (Cosine Similarity and Sentiment):


,product_name,similarity_score,sentiment_score,user_rating,evaluation_score
0,Breakfast Stout,0.018022,0.749884,4.5016,0.767906
1,Pseudo Sue,0.026180,0.711584,4.3884,0.737764
2,Dorothy (Wine Barrel Aged),0.044244,0.662536,4.6040,0.706780
3,Coffee Abraxas,0.000000,0.703236,4.5616,0.703236
4,Heady Topper,0.028753,0.672188,4.5864,0.700941
5,Framboise Du Fermier,0.017211,0.682004,4.5220,0.699215
6,Speedway Stout - Vietnamese Coffee,0.000000,0.681360,4.3244,0.681360
7,Cable Car,0.044997,0.632908,4.5772,0.677905
8,Ann,0.007476,0.661636,4.7076,0.669112
9,Trappist Westvleteren 8 (VIII),0.006554,0.662348,4.4884,0.668902


### Product Recommendation

In [15]:
# storing the top 3 products by evaluation score as recommended products and displaying them
recommended_products_cos = df_reviews_eval_cos['product_name'][:3].to_list()

print('\033[1mRecommended Products:\033[0m')
for i in recommended_products_cos:
    print(i)
    
print('\n\033[1mSimilarity, Sentiment Scores and User Ratings:\033[0m')
df_reviews_eval_cos[['product_name','similarity_score','sentiment_score','user_rating']].head(3)

Recommended Products:
Breakfast Stout
Pseudo Sue
Dorothy (Wine Barrel Aged)

Similarity, Sentiment Scores and User Ratings:


,product_name,similarity_score,sentiment_score,user_rating
0,Breakfast Stout,0.018022,0.749884,4.5016
1,Pseudo Sue,0.026180,0.711584,4.3884
2,Dorothy (Wine Barrel Aged),0.044244,0.662536,4.6040


## Task F: Recommendations using Word Vectors

In [16]:
# loading spaCy large word vector
nlp = spacy.load('en_core_web_lg')

### SpaCy Similarity (Vector) Score for each Review

In [17]:
# function to calculate spaCy similarity (vector) score
def spacy_sim(s):
    doc1 = nlp(user_attr_str)
    doc2 = nlp(s)
    return doc1.similarity(doc2)

# calculating spaCy similarity (vector) score for each review
df_reviews_vector = df_reviews_cosine.copy()
df_reviews_vector['similarity_score'] = df_reviews_vector['cleaned_review'].apply(spacy_sim)

# creating a copy DataFrame to calculate percentage count of attributes later
df_review_attr_count = df_reviews_vector.copy()

df_reviews_vector = df_reviews_vector[['product_name','product_review','similarity_score','sentiment_score','user_rating']]
df_reviews_vector = df_reviews_vector.sort_values(by = 'similarity_score', ascending = False)

# displaying the top 10 reviews by spaCy similarity (vector) score
print('Top 10 Reviews by SpaCy Similarity (Vector) Score:')
df_reviews_vector.head(10)

Top 10 Reviews by SpaCy Similarity (Vector) Score:


,product_name,product_review,similarity_score,sentiment_score,user_rating
4194,Saison Bernice,Tasted from bottle. Classic farmhouse saison. ...,0.776081,0.5106,4.44
1746,Ephraim,"Bright, refereshing, peppery, orange citrus, p...",0.767200,0.8863,4.60
1530,Citra,02/2015 - Pineapple and mango fosho with mild ...,0.761885,0.6908,4.44
2175,Galaxy Dry Hopped Fort Point Pale Ale,"11/2014 - Wow, this is amazing; incredible aro...",0.759170,0.9365,4.58
344,Pliny The Elder,"Bottle, kindly shared by gunnar. Deep golden t...",0.749732,0.8198,4.30
1057,Doubleganger,"Pours a medium golden color, very hazy. Ample ...",0.747982,0.5859,4.50
739,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,0.745153,0.6908,4.50
1199,Double Galaxy,"Soft gold, foamy head. Aroma has underripe mel...",0.740860,-0.0377,4.37
2122,A Deal With The Devil,"05/2015 - Love the huge cognac aromas, raisin ...",0.736256,0.8591,4.74
4260,Bad Boy,Shared from a growler with Ryan. Pours a prett...,0.727527,0.5927,4.25


### Vector Evaluation Scores for each Product

In [18]:
# calculating evaluation scores for each product
df_reviews_eval_vec = df_reviews_vector.copy()
df_reviews_eval_vec = df_reviews_eval_vec.groupby('product_name').mean()[['similarity_score','sentiment_score','user_rating']]
df_reviews_eval_vec['evaluation_score'] = df_reviews_eval_vec['similarity_score'] + df_reviews_eval_vec['sentiment_score']
df_reviews_eval_vec = df_reviews_eval_vec.sort_values(by = 'evaluation_score', ascending = False).reset_index()

# displaying the top 10 products by evaluation score
print('Top 10 Products by Evaluation Score (Vector Similarity and Sentiment):')
df_reviews_eval_vec.head(10)

Top 10 Products by Evaluation Score (Vector Similarity and Sentiment):


,product_name,similarity_score,sentiment_score,user_rating,evaluation_score
0,Breakfast Stout,0.447363,0.749884,4.5016,1.197247
1,Pseudo Sue,0.467542,0.711584,4.3884,1.179126
2,Speedway Stout - Vietnamese Coffee,0.470407,0.681360,4.3244,1.151767
3,Framboise Du Fermier,0.467074,0.682004,4.5220,1.149078
4,Coffee Abraxas,0.428922,0.703236,4.5616,1.132158
5,Trappist Westvleteren 8 (VIII),0.466308,0.662348,4.4884,1.128656
6,Beyond Good And Evil,0.463206,0.658492,4.4764,1.121698
7,Dorothy (Wine Barrel Aged),0.449303,0.662536,4.6040,1.111839
8,Tweak,0.464968,0.646584,4.4876,1.111552
9,Smooth,0.477017,0.631948,4.5296,1.108965


### Product Recommendation

In [19]:
# storing the top 3 products by vector evaluation score as recommended products and displaying them
recommended_products_vec = df_reviews_eval_vec['product_name'][:3].to_list()

print('\033[1mRecommended Products:\033[0m')
for i in recommended_products_vec:
    print(i)

print('\n\033[1mSimilarity, Sentiment Scores and User Ratings:\033[0m')
df_reviews_eval_vec[['product_name','similarity_score','sentiment_score','user_rating']].head(3)

Recommended Products:
Breakfast Stout
Pseudo Sue
Speedway Stout - Vietnamese Coffee

Similarity, Sentiment Scores and User Ratings:


,product_name,similarity_score,sentiment_score,user_rating
0,Breakfast Stout,0.447363,0.749884,4.5016
1,Pseudo Sue,0.467542,0.711584,4.3884
2,Speedway Stout - Vietnamese Coffee,0.470407,0.681360,4.3244


### User Attribute Mention Percentage

We check for percentage of mentions of user attributes in reviews to compare the different similarity scores.

In [20]:
# function to check for user attribute mentions in reviews
def attr_mention(s):
    for i in user_attr:
        if i in s:
            return 1
    return 0

# calculating percent of user attribute mentions in each review
df_review_attr_count['mention'] = df_review_attr_count['joined_tokens'].apply(attr_mention)
df_review_attr_count = df_review_attr_count.groupby('product_name')['mention'].agg(['sum','count'])
df_review_attr_count['mention_percent'] = df_review_attr_count['sum']/df_review_attr_count['count']
df_review_attr_count = df_review_attr_count[['mention_percent']].reset_index()

### Recommendation Comparison

In [21]:
# creating combined list of recommended beers for both features
recommended_products_all = recommended_products_cos + recommended_products_vec

# creating merged DataFrame containing similarity scores of both features
df_reviews_eval_merged = df_reviews_eval_cos.merge(df_reviews_eval_vec,
                                                   left_on = 'product_name',
                                                   right_on = 'product_name',
                                                   suffixes = ('_cos', '_vec'))\
                                            .drop(columns = ['sentiment_score_vec','user_rating_vec'])\
                                            .rename(columns = {'similarity_score_cos':'cosine_similarity_score',
                                                               'sentiment_score_cos':'sentiment_score',
                                                               'user_rating_cos':'user_rating',
                                                               'evaluation_score_cos':'cosine_evaluation_score',
                                                               'similarity_score_vec':'vector_similarity_score',
                                                               'evaluation_score_vec':'vector_evaluation_score'})\
                                            .merge(df_review_attr_count,
                                                   left_on = 'product_name',
                                                   right_on = 'product_name')

# filtering merged DataFrame for only the recommended products
df_reviews_eval = df_reviews_eval_merged[df_reviews_eval_merged['product_name'].isin(recommended_products_all)]
df_reviews_eval['cosine_recommended'] = df_reviews_eval['cosine_evaluation_score'].rank(ascending = False) <= 3
df_reviews_eval['vector_recommended'] = df_reviews_eval['vector_evaluation_score'].rank(ascending = False) <= 3

df_reviews_eval

,product_name,cosine_similarity_score,sentiment_score,user_rating,cosine_evaluation_score,vector_similarity_score,vector_evaluation_score,mention_percent,cosine_recommended,vector_recommended
0,Breakfast Stout,0.018022,0.749884,4.5016,0.767906,0.447363,1.197247,0.12,True,True
1,Pseudo Sue,0.026180,0.711584,4.3884,0.737764,0.467542,1.179126,0.12,True,True
2,Dorothy (Wine Barrel Aged),0.044244,0.662536,4.6040,0.706780,0.449303,1.111839,0.28,True,False
6,Speedway Stout - Vietnamese Coffee,0.000000,0.681360,4.3244,0.681360,0.470407,1.151767,0.08,False,True


Using the vector method, the three recommended beers are Breakfast Stout, Pseudo Sue, and Speedway Stout - Vietnamese Coffee. Using cosine-similarity, the three recommended beers are Breakfast Stout, Pseudo Sue, and Dorothy (Wine Barrel Aged). The third recommended beers are different among the two approaches while the first two are the same. Speedway Stout - Vietnamese Coffee had a cosine similarity score of zero, resulting in a lower evaluation score under this system. However, it had among the highest vector similarity and evaluation scores, resulting in it increasing to the third recommended beer in this system.  

This is justified by the different mechanics of the approaches. Cosine similarity takes into account only the mention of specific words (bag-of-words approach), whereas the vector approach takes context of words into account. This is further validated by the mention percentage ranking of Dorothy (Wine Barrel Aged). It has the highest mention percentage among the four beers being analyzed, and it was included as a recommended beer only in the cosine similarity approach due to this method taking specific words into account and the vector approach not taking into account the mention of only specific words.

## Task G: Recommendations using User Rating

### User Rating for each Product

In [22]:
df_reviews_ur = df_reviews_eval_merged[['product_name','user_rating']]
df_reviews_ur = df_reviews_ur.sort_values(by = 'user_rating', ascending = False)

# displaying the top 10 products by user rating
print('Top 10 Products by User Ratings:')
df_reviews_ur.head(10)

Top 10 Products by User Ratings:


,product_name,user_rating
119,Chemtrailmix,4.77160
135,Blessed,4.74880
24,Vanilla Bean Assassin,4.74625
44,SR-71,4.73960
151,It Was All A Dream,4.73600
13,Rare Scooop,4.73520
139,Kentucky Brunch Brand Stout,4.73480
8,Ann,4.70760
87,Marshmallow Handjee,4.69520
56,Art,4.69280


### Product Recommendation

In [23]:
# storing the top 3 products by user rating as recommended products and displaying them
recommended_products_ur = df_reviews_ur['product_name'][:3].to_list()

print('\033[1mRecommended Products:\033[0m')
for i in recommended_products_ur:
    print(i)

Recommended Products:
Chemtrailmix
Blessed
Vanilla Bean Assassin


### Recommendation Comparison

In [24]:
# adding new recommendations to the overall recommendations list
recommended_products_all = recommended_products_all + recommended_products_ur

# filtering merged DataFrame for only the recommended products
df_reviews_eval = df_reviews_eval_merged[df_reviews_eval_merged['product_name'].isin(recommended_products_all)]\
                                                                               .reset_index()\
                                                                               .drop(columns = 'index')

df_reviews_eval['cosine_recommended'] = df_reviews_eval['cosine_evaluation_score'].rank(ascending = False) <= 3
df_reviews_eval['vector_recommended'] = df_reviews_eval['vector_evaluation_score'].rank(ascending = False) <= 3
df_reviews_eval['user_rating_recommended'] = df_reviews_eval['user_rating'].rank(ascending = False) <= 3
df_reviews_eval

,product_name,cosine_similarity_score,sentiment_score,user_rating,cosine_evaluation_score,vector_similarity_score,vector_evaluation_score,mention_percent,cosine_recommended,vector_recommended,user_rating_recommended
0,Breakfast Stout,0.018022,0.749884,4.50160,0.767906,0.447363,1.197247,0.12,True,True,False
1,Pseudo Sue,0.026180,0.711584,4.38840,0.737764,0.467542,1.179126,0.12,True,True,False
2,Dorothy (Wine Barrel Aged),0.044244,0.662536,4.60400,0.706780,0.449303,1.111839,0.28,True,False,False
3,Speedway Stout - Vietnamese Coffee,0.000000,0.681360,4.32440,0.681360,0.470407,1.151767,0.08,False,True,False
4,Vanilla Bean Assassin,0.000000,0.634075,4.74625,0.634075,0.389370,1.023445,0.00,False,False,True
5,Chemtrailmix,0.000000,0.531320,4.77160,0.531320,0.401414,0.932734,0.00,False,False,True
6,Blessed,0.011122,0.501484,4.74880,0.512606,0.449579,0.951063,0.12,False,False,True


### Analysis  

Given that the sentiment scores of Vanilla Bean Assassin, Chemtrailmix, and Blessed are all lower than the previously recommended beers, we can begin to suspect this is a suboptimal recommendation system. Further analysis of the cosine similarity score shows that all three beers' scores are lower than the beers selected in the previous methods. Additional analysis of the vector similarity scores shows that these three recommended beers have lower scores compared to the previously recommended beers. Therefore, we can confidently say that the rankings of these three beers are lower than the other beers previously recommended, so these three beers are suboptimal to others in the eyes of the user.